In [1]:
### This is just a demonstartion script, full sampline is done at scale in ./hmatch

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import os
import pickle
import gpflow

import warnings
warnings.filterwarnings("ignore")

2025-04-13 07:50:52.548480: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 07:50:52.562425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744552252.573990   56154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744552252.577434   56154 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-13 07:50:52.591185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:

import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *
utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'
from sampling_pyfunctions import *

In [3]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='20GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    account='P93300041', # Input your project ID here
    walltime='02:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

### setup

In [4]:
# Define biome settings: biome name, index, and PFTs 
biome_configs = [
    {'name': 'NaN','index':0, 'pfts':[np.NaN]},
    {'name': 'Tropical rainforest', 'index':1, 'pfts': [4]},
    {'name': 'Tropical savanna', 'index':2, 'pfts': [4,14]},
    {'name': 'Subtropical savanna', 'index':3, 'pfts': [4,6,14]},
    {'name': 'Broadleaf evergreen temperate tree', 'index':4, 'pfts':[5, 13, 14]},
    {'name': 'Grasslands', 'index': 5, 'pfts':[13,14]},
    {'name': 'Shrubland', 'index':6, 'pfts': [10,13,14]},
    {'name': 'Mixed deciduous temperate forest', 'index':7, 'pfts': [1, 7, 13, 14]},
    {'name': 'Conifer forest', 'index':8, 'pfts': [1, 2, 13, 14]},
    {'name': 'Siberian larch', 'index':9, 'pfts': [3,11,12]},
    {'name': 'Boreal forest', 'index':10, 'pfts': [2,11,12]},
    {'name': 'Broadleaf deciduous boreal trees', 'index':11, 'pfts': [2, 8, 12, 13]},
    {'name': 'Boreal shrubland', 'index': 12, 'pfts': [12]},
    {'name': 'Tundra', 'index': 13, 'pfts': [11, 12]},
]

In [5]:
#############################################
# Setup
#############################################
# load observational data
obs = xr.open_dataset('../wave2_obsStatistics_sudokuBiomes.nc')

# info on parameter names
params_lhc = pd.read_csv('/glade/work/linnia/CLM6-PPE/ctsm6_lhc/ctsm6lhc_11262024.txt').drop(columns='member')

pft_params   = ['kmax','psi50','jmaxb0','slatop','lmr_intercept_atkin',
                'medlynslope','medlynintercept','froot_leaf','leafcn','leaf_long',
                'KCN','dleaf','r_mort','fsr_pft','xl']
pftix=np.array([p in pft_params for p in params_lhc.columns])
u_params = params_lhc.columns[~pftix]

pft_param_names = {i: [f"{param}_{i}" for param in pft_params] for i in range(1, 15)}

# Biome info
file='/glade/u/home/linnia/ctsm6_ppe/utils/sudoku_biomes.nc'
biomes=xr.open_dataset(file)
biome_names = biomes.biome_names.values

import pickle
with open("/glade/u/home/linnia/ctsm6_ppe/utils/biome_pft_dict.pkl", "rb") as f:
    biome_pfts = pickle.load(f)



### Sample, history match, save only NROY sets

In [5]:
# create universal sample
from scipy.stats import qmc
n_dims = 41
n_samples = 100000

sampler = qmc.LatinHypercube(d=n_dims)

s = sampler.random(n=n_samples)
usamples = pd.DataFrame(s,columns=u_params)

In [6]:
usamples.to_csv('./hmatch/universal_samples_LHC100000.csv',index=False)

In [9]:
usamples = pd.read_csv('universal_samples_LHC10000.csv')

In [24]:
%%time
emulator_path = '/glade/u/home/linnia/ctsm6_ppe/analysis_lhc/wave2_biome/emulators_biome'

n_psamp = 1000
key = np.array(1)
n_usets = 10

param_sets = []
for u in range(key*n_usets,key*n_usets+n_usets):
    usample = usamples.iloc[[u]]
    psample = pd.DataFrame(np.random.rand(n_psamp,len(pft_params)),columns=pft_params)

    b_samples = calibration_tree(usample,psample,n_psamp,u_params,pft_param_names,emulator_path,obs,biome_configs)
    # if none of the samples are plausible for any given biome, continue to next universal set
    if len(b_samples)==1:
        continue
    else:
        s = create_master_sample(b_samples,pft_param_names)
    
        param_sets.append(s)

master_sample = pd.concat(param_sets)

CPU times: user 2min 15s, sys: 1.29 s, total: 2min 17s
Wall time: 2min 26s


In [14]:
outdir = '/glade/work/linnia/CLM6-PPE/ctsm6_wave1/NROY/'
#master_sample.to_csv(out_dir+'hmatch_mastersample_'+str(key)+'.csv',index=False)
print(outdir+'hmatch_mastersample_'+str(key)+'.csv')

/glade/work/linnia/CLM6-PPE/ctsm6_wave1/NROY/hmatch_mastersample_1.csv
